In [ ]:
%load_ext autoreload
%autoreload 2

In [ ]:
import logging
import pickle
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import sep
import pandas as pd
import os
from astropy.table import Table, vstack

In [ ]:
from maddeb.utils import listdir_fullpath, get_data_dir_path

In [ ]:
logging.basicConfig(format="%(message)s", level=logging.INFO)
LOG = logging.getLogger(__name__)

## load the saved results

In [ ]:
density_level = "high"

In [ ]:
data_path = "/sps/lsst/users/bbiswas/MADNESS_results/" + density_level + "_density"
run_name = "kl01" # 12 to 20 galaxies
result_dir_path = os.path.join(data_path, run_name)

In [ ]:
madness_results=[]
scarlet_results=[]
actual_photometry = []
blended_photometry = []

for file_num in range(50):

    madness_results = pd.read_pickle(os.path.join(result_dir_path, "madness_results", str(file_num) + ".pkl"))

    actual_photometry = pd.read_pickle(os.path.join(data_path, "actual_photometry", str(file_num) + ".pkl"))
    blended_photometry = pd.read_pickle(os.path.join(data_path, "blended_photometry",  str(file_num) + ".pkl"))
    
    madness_results["r_contamination"] = ((blended_photometry["r_phot_flux"].values - actual_photometry["r_phot_flux"].values))/actual_photometry["r_phot_flux"].values

    madness_results.to_pickle(os.path.join(result_dir_path, "madness_results", str(file_num) + ".pkl"))

# Add magnitude values

In [ ]:
import hickle

simulation_path = os.path.join(
    "/sps/lsst/users/bbiswas/simulations/test_data/", density_level
)
mag_col_names = ['u_ab', 'g_ab', 'r_ab', 'i_ab', 'z_ab', 'y_ab']
num_repetations=300

for file_num in range(num_repetations):

    mag_data = []

    LOG.info("Processing file " + str(file_num))
    blend = hickle.load(
        os.path.join(
            simulation_path,
            str(file_num) + ".hkl",
        )
    )

    for field_num in range(len(blend["blend_list"])):
        df = blend["blend_list"][field_num].to_pandas()[mag_col_names]
        mag_data.append(df)
        
    mag_data = pd.concat(mag_data)

    madness_results = pd.read_pickle(os.path.join(result_dir_path, "madness_results", str(file_num) + ".pkl"))

    for col_name in mag_data.columns:
        madness_results[col_name] = mag_data[col_name].values
    
    madness_results.to_pickle(os.path.join(result_dir_path, "madness_results", str(file_num) + ".pkl"))

Processing file 0


KeyboardInterrupt: 